In [1]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
# from astropy.coordinates import EarthLocation
# from astropy.table import Table
# from astropy.time import Time
import astropy.units as u
# from icecream import ic, install
import numpy as np
import pandas as pd

import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

# install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


In [2]:
session = tlc.ObservingSession(ap.Observer.at_site("LBT"))
session.add_day_range("2025-05-31", "2025-06-04")

phase_defs = [
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.0)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.05)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.18)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.32)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.45)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.55)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.68)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.82)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.95)),
]

pepsi_args = {
    "fiber": "300",
    "cd_blue": 3,
    "cd_red": 6,
    "snr": 100,
    "binocular": True,
    "priority": "(see grid)",
}

name = "LBT Observing List 2025-05-31"
with connect("../../../../Data/astropaul.db") as conn:
    creator = tlc.TargetListCreator(name=name, connection=conn)
    creator.steps = [
        tlc.add_targets,
        partial(tlc.filter_targets, criteria=lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))),
        tlc.ancillary_data_from_tess,
        tlc.add_lists,
        partial(tlc.add_database_table, table_name="ephemerides"),
        partial(tlc.add_database_table, table_name="pepsi_observations"),
        partial(tlc.filter_targets, criteria=lambda df: (df["Num PEPSI Observations"] > 0)),
        partial(tlc.filter_targets, criteria=lambda df: (df["List LBT 2025A"])),
        lbt.add_rv_calibration_targets,
        partial(tlc.add_observability, observing_session=session, calc_moon_distance=True),
        partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
        # partial(tlc.filter_targets, criteria=lambda df: (df["Observable Max Alt"] > 40)),
        partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["Teff"].isna()),
        lbt.assign_rv_standards,
        partial(lbt.add_pepsi_params, **pepsi_args),
        partial(tlc.filter_targets, criteria=lambda df: df["PEPSI exp_time"] < 600),
        partial(tlc.add_rv_status, phase_event_defs=phase_defs),
    ]
    tl = creator.calculate(verbose=False)
tl.target_list["PEPSI notes"] = [type.replace("QuadEB", "Science Target") for type in tl.target_list["Target Type"]]
readme = lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session)

print(tl.summarize())
print(f"{np.sum(tl.target_list["PEPSI exp_time"])/60:.1f} minutes")
tl.target_list

Name: LBT Observing List 2025-05-31
Criteria
    lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))),
    lambda df: (df["Num PEPSI Observations"] > 0)),
    lambda df: (df["List LBT 2025A"])),
    Observability threshold: (<Quantity 30. deg>, <Quantity 80. deg>)
    AltitudeConstraint: {'min': np.float64(30.0), 'max': np.float64(80.0), 'boolean_constraint': True}
    lambda df: (df["Observable Any Night"])),
    Inverse of: lambda df: df["Teff"].isna()),
    lambda df: df["PEPSI exp_time"] < 600),
21 targets:
      13 QuadEB
       6 RV Standard
       2 Telluric Standard
Column Count (primary, secondary):
    Target: (3, 4)
    TESS Data: (2, 4)
    List: (0, 12)
    Count: (2, 0)
    Observable: (5, 20)
    PEPSI : (3, 6)
Associated tables:
     673 rows,  7 columns: Ephemerides
     181 rows, 13 columns: PEPSI Observations
       5 rows,  2 columns: Lunar Phases
     181 rows,  4 columns: PEPSI RV Status

51.6 minutes


,Target Name,RA,Dec,RA HMS,Dec DMS,Target Type,Source,PM RA,PM Dec,Vmag,...,RV Standard,PEPSI fiber,PEPSI cd_blue,PEPSI cd_blue_num_exp,PEPSI cd_red,PEPSI cd_red_num_exp,PEPSI snr,PEPSI exp_time,PEPSI priority,PEPSI notes
0,TIC 123098844,279.572833,44.698600,18:38:17.48,+44:41:54.96,QuadEB,Kostov 2022 arXiv:2202.05790,3.48250,-9.104270,11.136,...,BD +28 3402,300,3,1,6,1,100,189,(see grid),Science Target
15,TIC 204698586,185.551833,-24.224844,12:22:12.44,-24:13:29.44,QuadEB,Kostov 2022 arXiv:2202.05790,1.15779,5.322920,11.506,...,BD +28 3402,300,3,1,6,1,100,273,(see grid),Science Target
25,TIC 25818450,352.743458,53.069150,23:30:58.43,+53:04:08.94,QuadEB,Kostov 2022 arXiv:2202.05790,3.57360,-1.172130,11.782,...,HD 112299,300,3,1,6,1,100,341,(see grid),Science Target
35,TIC 278352276,307.503625,48.607056,20:30:00.87,+48:36:25.4,QuadEB,Kostov 2022 arXiv:2202.05790,1.85387,6.576640,10.387,...,HD 112299,300,3,1,6,1,100,94,(see grid),Science Target
36,TIC 283940788,8.851417,62.901594,00:35:24.34,+62:54:05.74,QuadEB,Kostov 2022 arXiv:2202.05790,-2.72664,-0.153866,11.774,...,HD 102870,300,3,1,6,1,100,328,(see grid),Science Target
44,TIC 307119043,14.827542,51.221642,00:59:18.61,+51:13:17.91,QuadEB,Kostov 2022 arXiv:2202.05790,7.56019,-4.108770,9.940,...,HD 102870,300,3,1,6,1,100,61,(see grid),Science Target
52,TIC 322727163,309.716625,50.466819,20:38:51.99,+50:28:00.55,QuadEB,Kostov 2022 arXiv:2202.05790,2.80000,-7.500000,10.997,...,HD 102870,300,3,1,6,1,100,154,(see grid),Science Target
64,TIC 389836747,23.295458,61.585308,01:33:10.91,+61:35:07.11,QuadEB,Kostov 2022 arXiv:2202.05790,-6.60000,-2.400000,10.718,...,HD 102870,300,3,1,6,1,100,116,(see grid),Science Target
69,TIC 414026507,336.837708,56.740361,22:27:21.05,+56:44:25.3,QuadEB,Kostov 2022 arXiv:2202.05790,-3.66449,-2.956470,10.899,...,HD 102870,300,3,1,6,1,100,137,(see grid),Science Target
72,TIC 427092089,321.112000,64.380683,21:24:26.88,+64:22:50.46,QuadEB,Kostov 2022 arXiv:2202.05790,-2.81497,-2.828370,12.367,...,HD 102870,300,3,1,6,1,100,530,(see grid),Science Target


In [3]:
illumination_categories = [
    ((0.0, 0.4), "Dark"),
    ((0.4, 0.7), "Gray"),
    ((0.7, 1.0), "Bright"),
]

distance_categories = {
    "Dark": [
        ((0, 180), 1),
    ],
    "Gray": [
        ((0, 5), 0.1),
        ((5, 15), 0.85),
        ((15, 180), 1),
    ],
    "Bright": [
        ((0, 15), 0.25),
        ((15, 30), 0.75),
        ((30, 180), 1),
    ],
}

altitude_categories = [
    ((-90, 35), 0),
    ((35, 45), 0.95),
    ((45, 90), 1),
]

phase_scores = {
    "B|B": 0.5,
    "B|R": 0.8,
    "B|T": 0.4,
    "R|B": 0.8,
    "R|R": 0.9,
    "R|T": 1,
    "T|B": 0.4,
    "T|R": 1,
    "T|T": 0.1,
}

science_tl = tl.copy()
science_tl.target_list = science_tl.target_list[science_tl.target_list["Target Type"] == "QuadEB"]
pl = pr.PriorityList(science_tl, session, interval=60 * u.min)
pr.calculate_moon_priority(pl, illumination_categories=illumination_categories, dist_categories=distance_categories)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.calculate_list_priority(pl, "HQND", false_value=0.75)
pr.calculate_phase_priority(pl, phase_defs=phase_defs, phase_categories=phase_scores)
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.3)
pl.categorize_priorities(
    bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"]
)

 '2025-05-31T05:00:00.000000000' '2025-05-31T06:00:00.000000000'
 '2025-05-31T07:00:00.000000000' '2025-05-31T08:00:00.000000000'
 '2025-05-31T09:00:00.000000000' '2025-05-31T10:00:00.000000000'
 '2025-05-31T11:00:00.000000000'], obsgeoloc=[(-5355335.07361679,  -344157.25148844, 3440995.87473266),
 (-5082228.0534148 , -1724335.62780892, 3440368.47301881),
 (-4460306.84776194, -2986354.65369275, 3438874.71137942),
 (-3532185.40964424, -4043740.95331699, 3436616.92784528),
 (-2361458.49084331, -4824042.55772083, 3433749.81204392),
 (-1028344.13809847, -5273793.30397522, 3430469.80674281),
 (  375812.83666948, -5362176.32947948, 3427001.64834419),
 ( 1754799.7905847 , -5083135.6392436 , 3423582.96852944),
 ( 3014128.72716687, -4455791.06185365, 3420448.01211214)] m, obsgeovel=[( 25.10492334, -391.13629014, -0.04856561),
 (125.74881265, -371.22096424, -0.29804689),
 (217.77641022, -325.8696278 , -0.52711014),
 (294.88199744, -258.1897483 , -0.72005988),
 (351.78230986, -172.81874195, -0.86

In [4]:
readme = lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session)
with open("UVa_Multistar.README", "w") as f:
    f.write(readme)

other_files = {
    "LBT Readme": lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session),
    "LBT CSV": open("UVa_Multistar.csv", "r").readlines(),
}

html.render_observing_pages(tl, pl, other_files, "../../../../Observing Files/LBT Observing 2025-05-31")
# pl.categorical_priorities[0]

In [ ]:
import glob
# for categorical_file in glob.glob("*.*"):

for categorical_file in glob.glob("../../Observing Files/LBT Observing 2025-03-14/Categorical Priorities *.html"):

    html.html_to_pdf(categorical_file, categorical_file.replace(".html", ".pdf"))
    print(categorical_file)

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.